In [1]:
import pkuseg
import re
from collections import Counter
import numpy as np

In [3]:
# 读取语料并分词
seg = pkuseg.pkuseg(user_dict ='./data/terminology8.txt')
with open('./data/korea_tai_vietn.txt',encoding = 'utf-8') as f:
    data = f.read()
data = re.sub('\n','',data)  #将换行符去掉
seg_list = seg.cut(data)
print('total words: ', len(seg_list))

total words:  103527


In [4]:
# 计算每个 n_gram 的概率
def n_gram_prob(seg_list, n):  #参数n是连续语块的词语数？
    prob = {}
    total_n_gram = []
    for i in range(len(seg_list) - n): 
        j = i + n
        gram = seg_list[i:j]
        is_chinese = [string >= u'\u4e00' and string <=u'\u9fa5' for string in gram]  # 保留只有中文字符的 n_gram
        if False not in is_chinese:  #判断是否是中文的，=找出是中文的
            total_n_gram.append(' '.join(gram))
    counter_dict = dict(Counter(total_n_gram))
    gram_num = len(total_n_gram)
    for key, value in counter_dict.items():
        prob[key] = value / gram_num
    return prob

In [5]:
# 计算New-MI
# 参数说明：seg_list为分好词的语料（列表格式），n为连续语块中包含的词语数，min_value为阈值
def new_mi(seg_list, n, min_value):
    results = {}
    n_gram_probs = []
    for i in range(n):
        n_gram_probs.append(n_gram_prob(seg_list, i+1))  
    
    for key, value in n_gram_probs[-1].items():
        gram = key.split()
        assert len(gram) == n   ##？？
        total_e = []
        for i in range(1, n):
            first_gram, second_gram = gram[:i], gram[i:]
            e = n_gram_probs[len(first_gram) - 1][' '.join(first_gram)] * n_gram_probs[len(second_gram) - 1][' '.join(second_gram)]
            total_e.append(e)
        wap = 0
        sum_e = sum(total_e)
        for e in total_e:
            wap += e * e/sum_e
        g = np.log(value/wap)
        if g >= min_value:
            results[key] = g
    return results
        

In [6]:
three_gram = new_mi(seg_list, 3, 3) # 3元组语块
four_gram = new_mi(seg_list, 4, 3) # 4元组语块
five_gram = new_mi(seg_list, 5, 3) # 5元组语块

In [9]:
print(type(three_gram),len(three_gram))

<class 'dict'> 34566


In [ ]:
with open('','w',encoding=)

In [7]:
three_gram

{'句 的 语义关系': 4.2621544594091345,
 '语义关系 非常 丰富': 7.820063815948048,
 '语法功能 相当 复杂': 6.4440110861226225,
 '是 汉语中 不可或缺': 4.515030044120914,
 '不可或缺 的 一': 5.1503767869453885,
 '一直 以 来': 5.726607313898683,
 '以 来 都': 3.804688840812764,
 '都 是 现代汉语语法': 6.102509372095351,
 '现代汉语语法 的 研究': 4.9918346933277045,
 '研究 重点 之一': 4.276533946040124,
 '本文 按照 郭圣林': 5.269627476824467,
 '下位 分类 再': 6.947567703883272,
 '分类 再 思考': 6.906341619972945,
 '一 文 中': 3.0390444196693567,
 '文 中 讨论': 4.217215457541792,
 '讨论 的 分类': 4.598660267565161,
 '分类 方式 对': 4.045549912466887,
 '出现 的 全部': 6.3666165367921,
 '然后 从 句式语义': 7.582384232252837,
 '从 句式语义 出发': 5.597415584329668,
 '句式语义 出发 逐类': 11.033592034489653,
 '出发 逐类 讨论': 7.588745888158485,
 '在 越南语 中': 3.2520849698545575,
 '越 译 特点': 4.102344224803631,
 '译 特点 并': 5.147150160571313,
 '特点 并 提出': 3.8408581302992,
 '并 提出 一些': 4.271626114590083,
 '提出 一些 越': 3.5457111905221934,
 '一些 越 译': 3.3530365614006836,
 '越 译 启发': 5.958521701431114,
 '可以 划分 为': 5.248631035457868,
 '这 些 小类': 4.51

In [61]:
four_gram

{'字 句 的 语义关系': 6.4638096300709496,
 '句 的 语义关系 非常': 6.2361466424552505,
 '汉语中 不可或缺 的 一': 6.396686879547574,
 '不可或缺 的 一 部分': 5.609460836150665,
 '一直 以 来 都': 6.018329225453481,
 '以 来 都 是': 4.793765434350222,
 '来 都 是 现代汉语语法': 5.5779345703128085,
 '是 现代汉语语法 的 研究': 4.721312795586329,
 '现代汉语语法 的 研究 重点': 8.05844153280902,
 '下位 分类 再 思考': 8.719584187230819,
 '一 文 中 讨论': 6.311111490402055,
 '中 讨论 的 分类': 4.794467837856323,
 '讨论 的 分类 方式': 6.9261786749602425,
 '中 出现 的 全部': 4.737572040822102,
 '然后 从 句式语义 出发': 7.684044509147998,
 '从 句式语义 出发 逐类': 5.5484530770687925,
 '句式语义 出发 逐类 讨论': 7.454576897682779,
 '越南语 中 的 表达': 4.044393618092849,
 '中 的 表达 形式': 4.762139829939493,
 '越 译 特点 并': 5.297771379511698,
 '译 特点 并 提出': 6.947121300324124,
 '特点 并 提出 一些': 6.124506816042355,
 '并 提出 一些 越': 5.317502745141275,
 '提出 一些 越 译': 6.202066415577415,
 '一些 越 译 启发': 5.979180648444935,
 '根据 这 些 小类': 7.249730119551323,
 '些 小类 的 百分比': 7.063064439707792,
 '小类 的 百分比 又': 7.793269703259645,
 '百分比 又 可以 将': 6.764037567803395,
 '又 可以 

In [62]:
five_gram

{'字 句 的 语义关系 非常': 6.760592187300764,
 '句 的 语义关系 非常 丰富': 6.3178871920119715,
 '是 汉语中 不可或缺 的 一': 4.692046258348565,
 '汉语中 不可或缺 的 一 部分': 5.643439286377298,
 '一直 以 来 都 是': 4.5884854326456015,
 '以 来 都 是 现代汉语语法': 6.329215312159984,
 '都 是 现代汉语语法 的 研究': 5.227375191581188,
 '是 现代汉语语法 的 研究 重点': 4.566359828233762,
 '现代汉语语法 的 研究 重点 之一': 7.808378034388435,
 '文 中 讨论 的 分类': 7.1481698109761,
 '中 讨论 的 分类 方式': 4.800360843287497,
 '讨论 的 分类 方式 对': 4.7252983763884915,
 '然后 从 句式语义 出发 逐类': 7.573151082864965,
 '从 句式语义 出发 逐类 讨论': 5.683605692590162,
 '在 越南语 中 的 表达': 4.806713953697508,
 '越南语 中 的 表达 形式': 5.888831475573547,
 '越 译 特点 并 提出': 6.1315058787752035,
 '译 特点 并 提出 一些': 6.903015222166598,
 '特点 并 提出 一些 越': 6.024321177808895,
 '并 提出 一些 越 译': 6.012535678911245,
 '提出 一些 越 译 启发': 6.829889784396817,
 '这 些 小类 的 百分比': 5.597788248964059,
 '些 小类 的 百分比 又': 7.240190971690226,
 '小类 的 百分比 又 可以': 6.31294255796154,
 '百分比 又 可以 将 它': 6.667293134569419,
 '又 可以 将 它 们': 5.980070106006677,
 '可以 将 它 们 归纳': 6.39446227203109,
 '将 它 